In [20]:
import pandas as pd
from tqdm import tqdm
import re

In [55]:
def Get_year_results(year_string, out_string):
    results = pd.read_html('https://sport.mkez.tw/games/'+ year_string +'/front/schedule_ball_score.php')[0]
    
    for i in tqdm(range(1,150)):
        results_temp = pd.read_html('https://sport.mkez.tw/games/'+ year_string +'/front/schedule_ball_score.php?pageNum_RS=' + str(i))[0]
        #print(html_tables_temp)
        results = pd.concat([results, results_temp])
        
    results.columns = ['matchID', 'Time', 'Group', 'inGroup_matchID', 'Player1', 'Score', 'Player2', 'Ranking']
    results = results.reset_index(drop = True)
    invalid_scores = ['傷', '空', '請', '棄', '-2', '-5']

    for i in invalid_scores:
        f = results['Score'].str.contains(i)
        results = results[~f]

    results['Gr'] = results['Group'].str.replace("子組第.次賽 圖", "", regex = True)
    results['Round'] = results['Group'].str.replace("(.子組| 圖)", "", regex = True)

    player_stats = {}
    for i in results.index:
        #player 1
        Player_entry = re.sub("(\\(.+\\)|[0-9]|勝)", "", results.Player1[i])
        Player_name = Player_entry.split(' - ')[1].strip() + ' - ' + Player_entry.split(' - ')[0]
    
        selection = '-'
        finals_app = 0
        score = results.Score[i].split(' : ')
        games_W = int(score[0])
        games_L = int(score[1])
        
        if games_W == 3:
            match_W = 1
            match_L = 0
            if '入選' in str(results.Ranking[i]):
                selection = results.Round[i] + '入選'
                finals_app = 1
        else:
            match_W = 0
            match_L = 1
            if '入選' in str(results.Ranking[i]):
                finals_app = 1
    
    
        if Player_name not in player_stats.keys():
            player_stats[Player_name] = {'name': Player_name, 'Gr': results.Gr[i],
                                         'W': match_W, 'L': match_L,
                                         'g_W': games_W, 'g_L': games_L, 
                                         'selection': selection, 'finals_app': finals_app
                                        }
        else:
            player_stats[Player_name]['W'] += match_W
            player_stats[Player_name]['L'] += match_L
            player_stats[Player_name]['g_W'] += games_W
            player_stats[Player_name]['g_L'] += games_L
            player_stats[Player_name]['finals_app'] += finals_app            

            if '入選' in selection:
                player_stats[Player_name]['selection'] = selection
        
        #player 2
        Player_entry = re.sub("(\\(.+\\)|[0-9]|勝)", "", results.Player2[i])
        Player_name = Player_entry.split(' - ')[1].strip() + ' - ' + Player_entry.split(' - ')[0]
        selection = '-'
        finals_app = 0
       
        score = results.Score[i].split(' : ')
        games_W = int(score[1])
        games_L = int(score[0])
    
        if games_W == 3:
            match_W = 1
            match_L = 0
            if '入選' in str(results.Ranking[i]):
                selection = results.Round[i] + '入選'
                finals_app = 1
        else:
            match_W = 0
            match_L = 1
            if '入選' in str(results.Ranking[i]):
                finals_app = 1
        
        if Player_name not in player_stats.keys():
            player_stats[Player_name] = {'name': Player_name, 'Gr': results.Gr[i],
                                         'W': match_W, 'L': match_L,
                                         'g_W': games_W, 'g_L': games_L,
                                         'selection': selection, 'finals_app': finals_app
                                        }
        else:
            player_stats[Player_name]['W'] += match_W
            player_stats[Player_name]['L'] += match_L
            player_stats[Player_name]['g_W'] += games_W
            player_stats[Player_name]['g_L'] += games_L
            player_stats[Player_name]['finals_app'] += finals_app            
            if '入選' in selection:
                player_stats[Player_name]['selection'] = selection

    player_stats = pd.DataFrame.from_dict(player_stats, orient='index').reset_index(drop = True)
    player_stats['Tot'] = player_stats.W + player_stats.L
    player_stats['W_pct'] = round(player_stats.W / player_stats.Tot, 3)
    player_stats['Tot_Gms'] = player_stats.g_W + player_stats.g_L
    player_stats['g_W_pct'] = round(player_stats.g_W / player_stats.Tot_Gms, 3)
    player_stats = player_stats.sort_values(by=['W_pct', 'g_W_pct'], ascending = False)
    player_stats = player_stats[['name', 'Gr', 
                                 'W', 'L', 'Tot',  'W_pct', 'selection', 'finals_app',
                                 'g_W', 'g_L',  'Tot_Gms', 'g_W_pct',]]
    
    player_stats.to_csv(out_string + '_results.tsv', sep="\t", index = False, float_format="%.3f") 
    
    player_stats_f = player_stats[player_stats['Gr'].str.contains('女')]
    player_stats_f.to_csv(out_string + '_results_f.tsv', sep="\t", index = False, float_format="%.3f") 
    
    player_stats_m = player_stats[player_stats['Gr'].str.contains('男')]
    player_stats_m.to_csv(out_string + '_results_m.tsv', sep="\t", index = False, float_format="%.3f") 

    return player_stats


In [56]:
Get_year_results('tt_11112', '112')

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 149/149 [00:40<00:00,  3.70it/s]


,name,Gr,W,L,Tot,W_pct,selection,finals_app,g_W,g_L,Tot_Gms,g_W_pct
214,李昱諄 - 國泰人壽,女,5,0,5,1.0,第一次賽入選,1,15,0,15,1.000
211,黎昕陽 - 第一銀行,男,6,0,6,1.0,第一次賽入選,1,18,3,21,0.857
213,高承睿 - 合作金庫,男,6,0,6,1.0,第一次賽入選,1,18,4,22,0.818
212,鄭先知 - 國泰人壽,女,5,0,5,1.0,第一次賽入選,1,15,5,20,0.750
208,劉馨尹 - 國泰人壽,女,9,1,10,0.9,第二次賽入選,2,29,6,35,0.829
...,...,...,...,...,...,...,...,...,...,...,...,...
125,鄭晴文 - 國泰人壽,女,0,6,6,0.0,-,0,1,18,19,0.053
179,陳楷庭 - 東泰高中,男,0,6,6,0.0,-,0,1,18,19,0.053
150,趙偉伯 - 彰化師大,男,0,2,2,0.0,-,0,0,6,6,0.000
161,蔡承翰 - 海山高中,男,0,4,4,0.0,-,0,0,12,12,0.000


In [59]:
Get_year_results('tt_1131112', '114')

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 149/149 [00:42<00:00,  3.50it/s]


,name,Gr,W,L,Tot,W_pct,selection,finals_app,g_W,g_L,Tot_Gms,g_W_pct
222,葉伊恬 - 合作金庫,女,6,0,6,1.0,第一次賽入選,1,18,3,21,0.857
221,蘇昱綸 - 合作金庫,男,6,0,6,1.0,第一次賽入選,1,18,4,22,0.818
220,吳映萱 - 國泰人壽,女,6,0,6,1.0,第一次賽入選,1,18,5,23,0.783
219,洪敬愷 - 第一銀行,男,6,0,6,1.0,第一次賽入選,1,18,7,25,0.720
216,黎彥君 - 第一銀行,男,9,1,10,0.9,第二次賽入選,1,28,10,38,0.737
...,...,...,...,...,...,...,...,...,...,...,...,...
143,虞善琦 -,男,0,5,5,0.0,-,0,0,15,15,0.000
145,顏瑜廷 - 個人報名,男,0,5,5,0.0,-,0,0,15,15,0.000
153,楊博翔 - 臺灣師大,男,0,1,1,0.0,-,0,0,3,3,0.000
155,林思佑 - 東山高中,男,0,5,5,0.0,-,0,0,15,15,0.000
